In [1]:
import csv
from download_h5py_db import download_file_from_google_drive
from tqdm import tqdm
from tensorflow.keras.models import load_model
from pathlib import Path
from utils import show_results
from sklearn.preprocessing import normalize
from numpy import asarray, argmax, bincount,argwhere
from h5py import File
from tqdm import tqdm
import numpy as np
from keras.utils import np_utils
from utils import create_dirs, SIZE, get_image_data, crop_and_save, font_to_num, is_not_dot


In [2]:
FILE_NAME = 'test_data.h5'
if not Path('test_data.h5').exists():
    download_file_from_google_drive('1YwLcXqLArFSOtoepQw7nC1t4jC8CFxpI', FILE_NAME)


In [3]:
path = 'res//top_model.h5'
if not Path('test_data.h5').exists():
    download_file_from_google_drive('TODO', path)
model = load_model(path)


In [6]:
def predict(model, db_file):
        db = File(db_file, 'r')   
        size=SIZE
        im_names = list(db['data'].keys())
        num = 0
        index = 0
        with open('test_lables.csv', 'w+', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([" ", "image", "char","Open Sans","Sansation","Titillium Web","Ubuntu Mono","Alex Brush"])
            for i in tqdm(range(0, len(im_names))):
                im = im_names[i]
                img, _, txts, charBBs, wordBBs = get_image_data(db, im, no_labels=True)
                font_indx = 0 
                char_indx = 0
                for j in range(0, len(txts)):
                    test_x = [] 
                    cropped = crop_and_save(img, wordBBs, j, size, None, im, num, True)
                    test_x.append(cropped)
                    num+=1            
                    for k in range(0, len(txts[j])):
                        if(is_not_dot(txts[j][k])):
                            word =  txts[j]
                            cropped = crop_and_save(img, charBBs, char_indx, size, None, im, num, True)
                            test_x.append(cropped)
                            num+=1
                            index += 1
                    test_x = asarray(test_x, dtype=np.float32)
                    reses = model.predict_on_batch(test_x)
                    maxes = np.argmax(reses, axis=1)
                    prediction = np.bincount(maxes)
                    prediction = np.argwhere(prediction==prediction.max())
                    if(len(prediction)>1):
                        sum_p = np.sum(reses, axis=0)
                        prediction = sum_p.argmax()
                    for k in range(0, len(word)):
                        a = []
                        a.append(index)
                        a.append(im)
                        a.append(chr(word[k]))
                        b = np_utils.to_categorical(prediction.item(), 5)
                        b = np.concatenate((a,b[1:],[b[0]]))
                        writer.writerow(b)
                        index+=1
                    font_indx += len(txts[j])

In [7]:
pred_y = predict(model, FILE_NAME)


100%|██████████| 1004/1004 [05:49<00:00,  2.87it/s]
